In [1]:
%cd ..

D:\Tesis\REPO\Cloud-Classification\code


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

In [4]:
import os
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx

In [5]:
from src import config
from src import engine
from src import utils

Using backend: pytorch


In [6]:
from recipe.utils import *

In [7]:
import random
from random import sample
from sklearn.metrics import accuracy_score

In [8]:
torch.manual_seed(7)
np.random.seed(7)
random.seed(7)

---

In [10]:
def get_targets(paths):
        return list(map(int,list(map(int,[os.path.basename(x).split('_')[0] 
                                          for x in paths]))))

In [72]:
def get_pivot_deep_features(paths, model, device):
    
    ## 1. train data loader
    pivot_dataset = GCD(paths, resize=256)
    pivot_loader = torch.utils.data.DataLoader(
            pivot_dataset,
            batch_size=32,
            num_workers=8,
            shuffle=False,
        )
    
    
    ## 2. Append deep features
    pivot_features = list()
    with torch.no_grad():    
        for data in pivot_loader:

            deep_features = model.get_deep_features(data['images'].to(device)).cpu()
            pivot_features.append(deep_features)
        
    return torch.cat(pivot_features, dim=0)

In [73]:
def get_top_indices(vector, matrix, k=5):
    with torch.no_grad():
        vector = vector/vector.norm()
        vector = vector.reshape(-1,1)

        norm = matrix.norm(dim=1).view(-1,1)
        matrix = matrix/norm

        scores = matrix@vector
        scores = scores.reshape(-1,)
    
    return scores.topk(k).indices

---

# 1. Leer df train y obtener deep features de train

In [74]:
paths = utils.get_gcd_paths('D:\Tesis\REPO','train')
targets = get_targets(paths)

In [75]:
df_train = pd.DataFrame({'path':paths, 'target':targets})

In [76]:
train_dfs = get_pivot_deep_features(df_train['path'].tolist(), model, device)

In [ ]:
## 1.1 Leer modelo

In [17]:
device='cuda'

model = GraphClassifier(7).to(device)
model.load_state_dict(torch.load(f'recipe/graph_params_AUGMENTATION.pt', map_location = 'cuda'))

model.eval()

GraphClassifier(
  (cnn): CNNExtractor(
    (cnn): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
     

## 1.2 Predicciones

In [68]:
path_test_images = utils.get_gcd_paths('D:\Tesis\REPO','test')

test_dataset = GCD(path_test_images, resize=256)

test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=1,
        num_workers=4,
        shuffle=False,
    )

In [83]:
df_eval = pd.DataFrame()

for k in [5,10,20,50,100]:
    print(f"--------- STARTING K={k} --------------")
    test_predictions = []
    test_targets = []
    for data in tqdm(test_loader):
            pred_img = data['images']

            ### get test df
            pred_feature = model.get_deep_features(pred_img.to(device)).cpu()

            ### get top k train indices
            indices = get_top_indices(pred_feature, train_dfs, k)

            ### pivot features are top k most similar
            pivot_features = train_dfs[indices]

            pred_images = torch.cat([pivot_features, pred_feature], dim=0).to(device)

            with torch.no_grad():
                preds = torch.argmax(F.softmax(model.message_passing(pred_images).cpu(),dim=1), dim=1)


                test_predictions.append(preds[-1].item())
                test_targets.append(data['targets'].item())
    
    acc = accuracy_score(test_targets, test_predictions)
    print(f"TEST ACC: {acc}", "\n")
    
    df_eval[f'prediction_K_{k}'] = test_predictions
    df_eval[f'target_K_{k}'] = test_targets

--------- STARTING K=5 --------------


100%|██████████████████████████████████████████████████████████████████████████████| 9010/9010 [10:13<00:00, 14.70it/s]


TEST ACC: 0.746503884572697 

--------- STARTING K=10 --------------


 43%|█████████████████████████████████▍                                            | 3861/9010 [04:18<05:41, 15.10it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000013F39C03430>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 1295, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000013F39C03430>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\dataloader

TEST ACC: 0.7473917869034407 

--------- STARTING K=20 --------------


...

TEST ACC: 0.7478357380688124 

--------- STARTING K=50 --------------


100%|██████████████████████████████████████████████████████████████████████████████| 9010/9010 [09:55<00:00, 15.12it/s]


TEST ACC: 0.74661487236404 

--------- STARTING K=100 --------------


100%|██████████████████████████████████████████████████████████████████████████████| 9010/9010 [10:16<00:00, 14.62it/s]

TEST ACC: 0.7472807991120977 



In [80]:
accuracy_score(test_targets, test_predictions)

0.746503884572697

In [85]:
df_eval.to_csv("inferencia_top_k_similar.csv", index=False)